In [5]:
# Advent of Code 2023
# Day 19 Problem 1
import re
from collections import Counter
import functools

with open("aoc_19_input.txt") as f:
   A =  f.read().strip()

# c+p test case here
TEST = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}""".strip()

In [32]:
from collections import defaultdict

# NOTE: come back and do this in a graph theoretic way in the future
def parse(inp):
    flows,raw_parts = [half.split("\n") for half in inp.strip().split("\n\n")]
    workflows = defaultdict()
    for flow in flows:
        # px{a<2006:qkq,m>2090:A,rfg}
        label, rules = flow[:-1].split("{")
        tmp = []
        for rule in rules.split(","):
            if ":" in rule:
                regex = re.search(r"(?P<categ>.*)(?P<comp>[><])(?P<quant>[0-9]+):(?P<nxt>.*)", rule)
                tmp.append([regex.group("categ"), regex.group("comp"), int(regex.group("quant")), regex.group("nxt")])
            else:
                tmp.append([rule])
        workflows[label] = tmp
    
    parts = defaultdict(lambda: defaultdict())
    for i, part in enumerate(raw_parts):
        for categ in part[1:-1].split(","):
            cat, quant = categ.split("=")
            parts[i][cat] = int(quant)
    
    return workflows, parts

def checkPart(workflows, part):
    currLabel = "in"
    while True:
        if currLabel == 'A':
            return 1
        if currLabel == 'R':
            return 0
        for flow in workflows[currLabel]:
            if flow == "A": 
                return 1
            if flow == "R":
                return 0
            if len(flow) == 1:
                currLabel = flow[0]
                break
            categ, comp, quant, nxt = flow
            #print(flow, part)
            if (comp == ">" and part[categ] > quant) or (comp == "<" and part[categ] < quant):
                currLabel = nxt
                break
            currLabel = nxt

def p1(inp):
    W, P = parse(inp)
    res = 0
    for idx in P:
        #print(P[idx])
        if checkPart(W, P[idx]):
            res += P[idx]['x'] + P[idx]['m'] + P[idx]['a'] + P[idx]['s']
    return res


In [33]:
# expected output: 19114
p1(TEST)

19114

In [34]:
p1(A)

353046

In [ ]:
def parseP2(inp):
    flows,_ = [half.split("\n") for half in inp.strip().split("\n\n")]
    workflows = defaultdict()
    for flow in flows:
        # px{a<2006:qkq,m>2090:A,rfg}
        label, rules = flow[:-1].split("{")
        tmp = []
        for rule in rules.split(","):
            if ":" in rule:
                regex = re.search(r"(?P<categ>.*)(?P<comp>[><])(?P<quant>[0-9]+):(?P<nxt>.*)", rule)
                tmp.append([regex.group("categ"), regex.group("comp"), int(regex.group("quant")), regex.group("nxt")])
            else:
                tmp.append([rule])
        workflows[label] = tmp
    
    return workflows

def split_ranges(uncertain):
    acceptedRanges = []
    rejectedRanges = []
    


def p2(inp):
    workflows = parseP2(inp)
    uncertainRanges = [([1,4000], [1,4000], [1,4000], [1,4000])]
    return split_ranges(uncertainRanges)

